In [1]:
'''
在构造NN模块时，我们需要达到以下几个目的：
1 设置选项
2 注册可学习的参数或者子模块
3 初始化参数
'''

'\n在构造NN模块时，我们需要达到以下几个目的：\n1 设置选项\n2 注册可学习的参数或者子模块\n3 初始化参数\n'

In [5]:
import torch.nn as nn
from dgl.utils import expand_as_pair
import dgl.function as fn
import torch.nn.functional as F
from dgl.utils import check_eq_shape

在构造函数中，用户首先需要设置数据的维度。  
对于一般的PyTorch模块，维度通常包括输入的维度、输出的维度和隐层的维度。  
对于图神经网络，输入维度可被分为**源节点特征维度和目标节点特征维度**。

除了数据维度，图神经网络的一个典型选项是**聚合类型**(self._aggre_type)。  
对于特定目标节点，聚合类型决定了如何聚合不同边上的信息。   
常用的聚合类型包括 mean、 sum、 max 和 min。一些模块可能会使用更加复杂的聚合函数，比如 lstm。

In [8]:
class SAGEConv(nn.Module):
    '''设置选项'''
    def __init__(self,
                 #输入
                 in_feats,
                 #输出
                 out_feats,
                 #聚合类型
                 aggregator_type,
                 bias=True,
                 #特征归一化
                 norm=None,
                 activation=None):
        super(SAGEConv, self).__init__()
        
        #将输入拆分为源节点和目标节点
        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats)
        self._out_feats = out_feats
        self._aggre_type = aggregator_type
        self.norm = norm
        self.activation = activation
        
        # 聚合类型：mean、pool、lstm、gcn
        if aggregator_type not in ['mean', 'pool', 'lstm', 'gcn']:
            raise KeyError('Aggregator type {} not supported.'.format(aggregator_type))
        if aggregator_type == 'pool':
            self.fc_pool = nn.Linear(self._in_src_feats, self._in_src_feats)
        if aggregator_type == 'lstm':
            self.lstm = nn.LSTM(self._in_src_feats, self._in_src_feats, batch_first=True)
        if aggregator_type in ['mean', 'pool', 'lstm']:
            self.fc_self = nn.Linear(self._in_dst_feats, out_feats, bias=bias)
        self.fc_neigh = nn.Linear(self._in_src_feats, out_feats, bias=bias)
        self.reset_parameters()
        
        
    '''注册参数和子模块'''
    def reset_parameters(self):
        """重新初始化可学习的参数"""
        gain = nn.init.calculate_gain('relu')
        if self._aggre_type == 'pool':
            nn.init.xavier_uniform_(self.fc_pool.weight, gain=gain)
        if self._aggre_type == 'lstm':
            self.lstm.reset_parameters()
        if self._aggre_type != 'gcn':
            nn.init.xavier_uniform_(self.fc_self.weight, gain=gain)
        nn.init.xavier_uniform_(self.fc_neigh.weight, gain=gain)
        
        
    '''forward函数'''
    def forward(self, graph, feat):
        with graph.local_scope():
            # 指定图类型，然后根据图类型扩展输入特征
            feat_src, feat_dst = expand_as_pair(feat, graph)
            
        '''消息传递和聚合'''
        if self._aggre_type == 'mean':
            graph.srcdata['h'] = feat_src
            graph.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'neigh'))
            h_neigh = graph.dstdata['neigh']
        elif self._aggre_type == 'gcn':
            check_eq_shape(feat)
            graph.srcdata['h'] = feat_src
            graph.dstdata['h'] = feat_dst
            graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'neigh'))
            # 除以入度
            degs = graph.in_degrees().to(feat_dst)
            h_neigh = (graph.dstdata['neigh'] + graph.dstdata['h']) / (degs.unsqueeze(-1) + 1)
        elif self._aggre_type == 'pool':
            graph.srcdata['h'] = F.relu(self.fc_pool(feat_src))
            graph.update_all(fn.copy_u('h', 'm'), fn.max('m', 'neigh'))
            h_neigh = graph.dstdata['neigh']
        else:
            raise KeyError('Aggregator type {} not recognized.'.format(self._aggre_type))

        # GraphSAGE中gcn聚合不需要fc_self
        if self._aggre_type == 'gcn':
            rst = self.fc_neigh(h_neigh)
        else:
            rst = self.fc_self(h_self) + self.fc_neigh(h_neigh)
        
        '''聚合后，更新特征作为输出'''
        # 激活函数
        if self.activation is not None:
            rst = self.activation(rst)
        # 归一化
        if self.norm is not None:
            rst = self.norm(rst)
        return rst